In [43]:
import os
import json
import pandas as pd
import yfinance as yf

ticker = "COIN"
# 1) Fetch the quarterly income statement
df = yf.Ticker(ticker).quarterly_income_stmt

# 2) Pick your three metrics and pivot so dates → rows
metrics = ["Total Revenue", "Operating Revenue", "EBITDA"]
subset = df.loc[metrics].T

# 3) Index as ISO‑strings
subset.index = subset.index.strftime("%Y-%m-%d")

# 4) Convert any NaN → None
subset = subset.where(pd.notnull(subset), None)

# 5) To nested dict { date: { metric: value, … }, … }
data = subset.to_dict(orient="index")

# 6) Ensure output dir exists
os.makedirs("yahoofinance-raw-data", exist_ok=True)

# 7) Dump valid JSON (None → null)
out_path = f"yahoofinance-raw-data/{ticker}_quarterly_metrics.json"
with open(out_path, "w") as f:
    json.dump(data, f, indent=2)

print(f"Wrote {out_path}")

Wrote yahoofinance-raw-data/COIN_quarterly_metrics.json


In [44]:
import os
import json
import yfinance as yf

# User parameters
ticker     = "COIN"
start_date = "2024-09-01"
end_date   = "2025-04-21"

# 1) Fetch shares outstanding
tkr = yf.Ticker(ticker)
shares_out = tkr.info.get("sharesOutstanding")
if shares_out is None:
    raise ValueError("Could not fetch sharesOutstanding from yfinance")

# 2) Download history and compute MarketCap
hist = tkr.history(start=start_date, end=end_date)
hist["MarketCap"] = hist["Close"] * shares_out

# 3) Prepare a plain dict { "YYYY‑MM‑DD": marketCap, … }
mc_df = hist["MarketCap"].reset_index()
mc_df["Date"] = mc_df["Date"].dt.strftime("%Y-%m-%d")
market_caps = {row["Date"]: row["MarketCap"] for _, row in mc_df.iterrows()}

# 4) Ensure output directory exists
os.makedirs("yahoofinance-raw-data", exist_ok=True)

# 5) Save to JSON
out_path = f"yahoofinance-raw-data/{ticker}_market_cap.json"
with open(out_path, "w") as f:
    json.dump(market_caps, f, indent=2)

print(f"Wrote market caps to {out_path}")

Wrote market caps to yahoofinance-raw-data/COIN_market_cap.json


In [50]:
import os
import json
import bisect
import math
from datetime import datetime
import yfinance as yf
import pandas as pd

# --- User parameters ---
ticker        = "COIN"
metrics_file  = f"yahoofinance-raw-data/{ticker}_quarterly_metrics.json"
output_file   = f"yahoofinance-raw-data/{ticker}_mcap_to_oprev_multiples.json"

# 1) Load the quarterly metrics
with open(metrics_file, "r") as f:
    quarterly_metrics = json.load(f)

# 2) Sort quarter‑end dates
quarter_strs  = sorted(quarterly_metrics.keys())
quarter_dates = [datetime.strptime(q, "%Y-%m-%d") for q in quarter_strs]

# 3) Fetch daily history and compute market cap
tkr = yf.Ticker(ticker)
hist = tkr.history(start=quarter_strs[0],
                   end=datetime.today().strftime("%Y-%m-%d"))
shares_out = tkr.info.get("sharesOutstanding")
if shares_out is None:
    raise ValueError("Could not fetch sharesOutstanding")

# strip timezone so comparisons work
if hasattr(hist.index, "tz"):
    hist.index = hist.index.tz_localize(None)

hist["MarketCap"] = hist["Close"] * shares_out

# 4) Build daily_data: market_cap, operating_revenue, multiple
daily_data = {}
for dt, row in hist.iterrows():
    dt_py    = dt.to_pydatetime()
    date_str = dt_py.strftime("%Y-%m-%d")
    mcap     = row["MarketCap"]

    # find most recent quarter ≤ date
    idx = bisect.bisect_right(quarter_dates, dt_py) - 1
    if idx < 0:
        continue
    quarter = quarter_strs[idx]
    op_rev  = quarterly_metrics[quarter]["Operating Revenue"]
    if op_rev is None or op_rev == 0:
        continue

    daily_data[date_str] = {
        "market_cap": mcap,
        "operating_revenue": op_rev,
        "mcap_to_oprev": mcap / op_rev
    }

# 5) Compute trailing 7‑day average of the multiple
sorted_dates = sorted(daily_data.keys())
vals         = [daily_data[d]["mcap_to_oprev"] for d in sorted_dates]
trailing_avg = {}
for i, d in enumerate(sorted_dates):
    if i < 6:
        trailing_avg[d] = None
    else:
        window = vals[i-6 : i+1]
        trailing_avg[d] = sum(window) / len(window)

# 6) Merge into final dict
results = {}
for d in sorted_dates:
    results[d] = {
        "market_cap": daily_data[d]["market_cap"],
        "operating_revenue": daily_data[d]["operating_revenue"],
        "mcap_to_oprev": daily_data[d]["mcap_to_oprev"],
        "7d_avg": trailing_avg[d]
    }

# 7) Sanitize any remaining NaN → None
for date, metrics in results.items():
    for key, val in metrics.items():
        if isinstance(val, float) and math.isnan(val):
            results[date][key] = None

# 8) (Optional) Print for manual inspection
df = pd.DataFrame.from_dict(results, orient="index")
print(df)

# 9) Ensure output directory exists and save JSON
os.makedirs(os.path.dirname(output_file) or ".", exist_ok=True)
with open(output_file, "w") as f:
    json.dump(results, f, indent=2)

print(f"Wrote sanitized market-cap/operating-revenue multiples to {output_file}")


              market_cap  operating_revenue  mcap_to_oprev     7d_avg
2024-01-02  3.296912e+10       9.046380e+08      36.444543        NaN
2024-01-03  3.199400e+10       9.046380e+08      35.366632        NaN
2024-01-04  3.270012e+10       9.046380e+08      36.147189        NaN
2024-01-05  3.235967e+10       9.046380e+08      35.770847        NaN
2024-01-08  3.350291e+10       9.046380e+08      37.034605        NaN
...                  ...                ...            ...        ...
2025-04-14  3.710917e+10       2.197030e+09      16.890608  15.961806
2025-04-15  3.689692e+10       2.197030e+09      16.793997  16.167052
2025-04-16  3.619080e+10       2.197030e+09      16.472600  16.371069
2025-04-17  3.678343e+10       2.197030e+09      16.742343  16.693013
2025-04-21  3.677713e+10       2.197030e+09      16.739474  16.664454

[326 rows x 4 columns]
Wrote sanitized market-cap/operating-revenue multiples to yahoofinance-raw-data/COIN_mcap_to_oprev_multiples.json
